##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quantization aware training in Keras example

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/quantization/training_example"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/g3doc/guide/quantization/training_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/g3doc/guide/quantization/training_example.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/model-optimization/tensorflow_model_optimization/g3doc/guide/quantization/training_example.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

Welcome to an end-to-end example for *quantization aware training*.

### Other pages
For an introduction to what quantization aware training is and to determine if you should use it (including what's supported), see the [overview page](https://www.tensorflow.org/model_optimization/guide/quantization/training.md).

To quickly find the APIs you need for your use case (beyond fully-quantizing a model with 8-bits), see the
[comprehensive guide](https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide.md).

### Summary

In this tutorial, you will:

1.   Train a `keras` model for MNIST from scratch.
2.   Fine tune the model by applying the quantization aware training API, see the accuracy, and
     export a quantization aware model.
3.   Use the model to create an actually quantized model for the TFLite
     backend.
4.   See the persistence of accuracy in
     TFLite and a 4x smaller model. To see the latency benefits on mobile, try out the TFLite examples [in the TFLite app repository](https://www.tensorflow.org/lite/models).

## Setup

In [1]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.5 MB/s eta 0:00:00


In [2]:
import tempfile
import os

import tensorflow as tf

from tensorflow_model_optimization.python.core.keras.compat import keras

In [3]:
! pip show tensorflow

Name: tensorflow
Version: 2.17.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras


In [4]:
! pip install keras==2.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires keras>=3.2.0, but you have keras 2.11.0 which is incompatible.


## Train a model for MNIST without quantization aware training

In [ ]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=1,
  validation_split=0.1,
)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import tensorflow as tf

model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/finalmodes/5devices_mobilenet_pos1" ,compile=False)

In [20]:
import tensorflow as tf

model = tf.saved_model.load('/content/drive/MyDrive/Colab Notebooks/finalmodes/5devices_cnn_pos1_2')


TypeError: load() got an unexpected keyword argument 'compile'

In [23]:
from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/Colab Notebooks/finalmodes/5devices_cnn_pos1", compile=False)

ValueError: File format not supported: filepath=/content/drive/MyDrive/Colab Notebooks/finalmodes/5devices_cnn_pos1. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(/content/drive/MyDrive/Colab Notebooks/finalmodes/5devices_cnn_pos1, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

## Clone and fine-tune pre-trained model with quantization aware training


### Define the model

You will apply quantization aware training to the whole model and see this in the model summary. All layers are now prefixed by "quant".

Note that the resulting model is quantization aware but not quantized (e.g. the weights are float32 instead of int8). The sections after show how to create a quantized model from the quantization aware one.

In the [comprehensive guide](https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide.md), you can see how to quantize some layers for model accuracy improvements.

In [7]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 102, 22, 1)]      0         
                                                                 
 quantize_layer (QuantizeLa  (None, 102, 22, 1)        3         
 yer)                                                            
                                                                 
 quant_conv2d (QuantizeWrap  (None, 51, 11, 32)        385       
 perV2)                                                          
                                                                 
 quant_batch_normalization   (None, 51, 11, 32)        129       
 (QuantizeWrapperV2)                                             
                                                                 
 quant_re_lu (QuantizeWrapp  (None, 51, 11, 32)        3         
 erV2)                                                       

In [8]:
import os
path="/content/drive/MyDrive/Colab Notebooks"
os.chdir(path)
os.listdir(path)

['Untitled1.ipynb',
 '.ipynb_checkpoints',
 'textcnn',
 'unzipfile.ipynb',
 'text-classification-cnn-rnn-master.zip',
 'cnnBYpytorch.ipynb',
 'text-classification-cnn-rnn-master',
 'Test.ipynb',
 'cnn-rnn',
 'Untitled0.ipynb',
 'finalmodes',
 '__pycache__',
 'dataset_preparation.py']

In [9]:
from dataset_preparation import awgn, LoadDataset, ChannelIndSpectrogram

### Train and evaluate the model against baseline

To demonstrate fine tuning after training the model for just an epoch, fine tune with quantization aware training on a subset of the training data.

In [12]:
from dataset_preparation import awgn, LoadDataset, ChannelIndSpectrogram
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import numpy as np
LoadDatasetObj = LoadDataset()
data, label=LoadDatasetObj.load_data_lable_train()
print("data shape label shape:",data.shape,label.shape)
snr_range = np.arange(20,80)
data = awgn(data, snr_range)

ChannelIndSpectrogramObj = ChannelIndSpectrogram()

# Convert time-domain IQ samples to channel-independent spectrograms.
data = ChannelIndSpectrogramObj.channel_ind_spectrogram(data)
batch_size = 32
patience = 20


early_stop = EarlyStopping('val_loss',
                               min_delta = 0,
                               patience =
                               patience)

reduce_lr = ReduceLROnPlateau('val_loss',
                              min_delta = 0,
                              factor = 0.2,
                              patience = 10,
                              verbose=1)
callbacks = [early_stop, reduce_lr]

# Split the dasetset into validation and training sets.
data_train, data_valid, label_train, label_valid = train_test_split(data, label, test_size=0.1, shuffle= True)
print("data_train, data_valid, label_train, label_valid:",data_train.shape, data_valid.shape, label_train.shape, label_valid.shape)
del data, label

(18000, 3180) (1, 18000)
data shape label shape: (18000, 3180) (18000, 1)
num_column:  22
(18000, 102, 22, 1)
data_train, data_valid, label_train, label_valid: (16200, 102, 22, 1) (1800, 102, 22, 1) (16200, 1) (1800, 1)


In [ ]:
q_aware_model.fit(x=data_train,y=label_train,steps_per_epoch = data_train.shape[0]//batch_size,
                              epochs = 100,
                              validation_data = (data_valid,label_valid),
                              validation_steps = data_valid.shape[0]//batch_size,
                              verbose=1)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/tf_keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


506/506 [==============================] - ETA: 0s - loss: 0.0897 - accuracy: 0.9835

506/506 [==============================] - 336s 648ms/step - loss: 0.0897 - accuracy: 0.9835 - val_loss: 0.0858 - val_accuracy: 0.9833
Epoch 2/100
506/506 [==============================] - 293s 579ms/step - loss: 0.0225 - accuracy: 0.9953
Epoch 3/100
506/506 [==============================] - 283s 560ms/step - loss: 0.0024 - accuracy: 0.9994
Epoch 4/100
506/506 [==============================] - 276s 546ms/step - loss: 8.4271e-04 - accuracy: 0.9999
Epoch 5/100
506/506 [==============================] - 279s 551ms/step - loss: 3.9756e-04 - accuracy: 0.9999
Epoch 6/100
506/506 [==============================] - 277s 548ms/step - loss: 0.0055 - accuracy: 0.9987
Epoch 7/100
506/506 [==============================] - 281s 555ms/step - loss: 0.0347 - accuracy: 0.9892
Epoch 8/100
506/506 [==============================] - 276s 546ms/step - loss: 0.0182 - accuracy: 0.9940
Epoch 9/100
506/506 [==============================] - 277s 548ms/step - loss: 0.0059 - accuracy: 0.9980
Epoch 10/100
506/

In [ ]:

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=200, epochs=1, validation_split=0.1)

For this example, there is minimal to no loss in test accuracy after quantization aware training, compared to the baseline.

In [ ]:
data_enrol, true_label=LoadDatasetObj.load_iq_samples_test()

_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

## Create quantized model for TFLite backend

After this, you have an actually quantized model with int8 weights and uint8 activations.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

## See persistence of accuracy from TF to TFLite

Define a helper function to evaluate the TF Lite model on the test dataset.

In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

You evaluate the quantized model and see that the accuracy from TensorFlow persists to the TFLite backend.

In [ ]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

## See 4x smaller model from quantization

You create a float TFLite model and then see that the quantized TFLite model
is 4x smaller.

In [ ]:
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

## Conclusion

In this tutorial, you saw how to create quantization aware models with the TensorFlow Model Optimization Toolkit API and then quantized models for the TFLite backend.

You saw a 4x model size compression benefit for a model for MNIST, with minimal accuracy
difference. To see the latency benefits on mobile, try out the TFLite examples [in the TFLite app repository](https://www.tensorflow.org/lite/models).

We encourage you to try this new capability, which can be particularly important for deployment in resource-constrained environments.
